In [1]:
import pandas as pd

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
import string
import re

In [6]:
!ls

Raul_scratch_code.ipynb


In [7]:
cd ..

/Users/raulcassieltorresponce/Documents/Twitter_NLP_analysis


In [9]:
!ls

Data         Images       README.md    Scratch_code


In [11]:
df = pd.read_csv('data/judge-1377884607_tweet_product_company.csv', encoding='Latin-1')
df.tail(10)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
9083,"Google says the future is all around you! (ie,...",NaN,No emotion toward brand or product
9084,"Google says the future is location, location, ...",NaN,No emotion toward brand or product
9085,I've always used Camera+ for my iPhone b/c it ...,iPad or iPhone App,Positive emotion
9086,Google says: want to give a lightning talk to ...,NaN,No emotion toward brand or product
9087,"@mention Yup, but I don't have a third app yet...",NaN,No emotion toward brand or product
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product
9092,Ï¡Ïàü_ÊÎÒ£Áââ_£â_ÛâRT @...,NaN,No emotion toward brand or product


In [12]:
df['tweet_text'].isna().sum()

1

In [13]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [14]:
df['emotion_in_tweet_is_directed_at'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [15]:
# replace iPhone to Apple
df['emotion_in_tweet_is_directed_at'].replace(to_replace='iPhone', value='Apple', inplace=True)

In [16]:
# replace ipad or iphone app to apple 
df['emotion_in_tweet_is_directed_at'].replace(to_replace='iPad or iPhone App', value='Apple', inplace=True)

In [17]:
# replace ipad or iphone app to apple 
df['emotion_in_tweet_is_directed_at'].replace(to_replace='iPad', value='Apple', inplace=True)

In [18]:
# replace ther Apple pr to apple 
df['emotion_in_tweet_is_directed_at'].replace(to_replace='Other Apple product or service', value='Apple', inplace=True)

In [19]:
df['emotion_in_tweet_is_directed_at'].value_counts()

Apple                              2409
Google                              430
Other Google product or service     293
Android App                          81
Android                              78
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [20]:
# replace Android App to Google
df['emotion_in_tweet_is_directed_at'].replace(to_replace='Android App', value='Google', inplace=True)

In [21]:
# replace ther Google product or service App to Google
df['emotion_in_tweet_is_directed_at'].replace(to_replace='Other Google product or service', value='Google', inplace=True)

In [22]:
# replace "Android" to Google
df['emotion_in_tweet_is_directed_at'].replace(to_replace='Android', value='Google', inplace=True)

In [23]:
df['emotion_in_tweet_is_directed_at'].value_counts()

Apple     2409
Google     882
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [24]:
X = df['tweet_text']
y = df['is_there_an_emotion_directed_at_a_brand_or_product']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

NameError: name 'train_test_split' is not defined

In [25]:
sample_document = df['tweet_text'][1]
sample_document

"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW"

In [26]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern)
sample_doc = tokenizer.tokenize(sample_document)
sample_doc = [token.lower() for token in sample_doc]
sw = stopwords.words('english')
sample_doc = [token for token in sample_doc if token not in sw]


In [27]:
sample_doc

['jessedee',
 'know',
 'fludapp',
 'awesome',
 'ipad',
 'iphone',
 'app',
 'likely',
 'appreciate',
 'design',
 'also',
 "they're",
 'giving',
 'free',
 'ts',
 'sxsw']

In [29]:
# dropping the null value
df.dropna(subset=['tweet_text'], how='all', inplace=True)

In [30]:
# checking it has been dropped
df['tweet_text'].isna().sum()

0

In [32]:
# clean the text
# create a function
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  #remove @ mentions
    text = re.sub(r'#', '', text)   #remove the '# symbol'
    text = re.sub(r'RT[\s]+', '', text)   #remove RT retweet
    text = re.sub(r'https?:\/\/\S+', '', text)  #remove the hyper link
    
    return text

#cleaning the text
df['tweet_text'] = df['tweet_text'].apply(cleanTxt)

In [66]:
# We removed numbers and punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['tweet_text'] = df['tweet_text'].apply(lambda x: remove_punct(x))

In [50]:
# @Terry: replaced Nan from second column to Apple or Google adding it to a fourth column
def Find_Apple_or_Google(text):
    category =''
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    tokenizer = RegexpTokenizer(pattern)
    sample_doc = tokenizer.tokenize(text)
    sample_doc = [token.lower() for token in sample_doc]
    sw = stopwords.words('english')
    sample_doc = [token for token in sample_doc if token not in sw]
    ap = ['apple', 'ipad', 'iphone']
    go = ['google','android']
    if any((match := item) in sample_doc for item in ap):
        return 'Apple'
    elif any((match := item) in sample_doc for item in go):
        return 'Google'
    else:
        return 'NAN'

In [51]:
df['Product_Category'] = df['tweet_text'].apply(lambda x: Find_Apple_or_Google(x))

In [52]:
df['Product_Category'].value_counts() 

Apple     5455
Google    2614
NAN       1023
Name: Product_Category, dtype: int64

In [75]:
df.columns

Index(['tweet_text', 'emotion_in_tweet_is_directed_at',
       'is_there_an_emotion_directed_at_a_brand_or_product',
       'Product_Category', 'Tweet_punct'],
      dtype='object')

In [78]:
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,Product_Category,Tweet_punct
0,I have a G iPhone After hrs tweeting at RISE...,Apple,Negative emotion,Apple,I have a G iPhone After hrs tweeting at RISE...
1,Know about Awesome iPadiPhone app that youl...,Apple,Positive emotion,Apple,Know about Awesome iPadiPhone app that youl...
2,Can not wait for iPad also They should sale ...,Apple,Positive emotion,Apple,Can not wait for iPad also They should sale ...
3,I hope this years festival isnt as crashy as ...,Apple,Negative emotion,Apple,I hope this years festival isnt as crashy as ...
4,great stuff on Fri SXSW Marissa Mayer Google ...,Google,Positive emotion,Google,great stuff on Fri SXSW Marissa Mayer Google ...
...,...,...,...,...,...
9088,Ipad everywhere SXSW link,Apple,Positive emotion,Apple,Ipad everywhere SXSW link
9089,Wave buzz We interrupt your regularly schedule...,NaN,No emotion toward brand or product,Google,Wave buzz We interrupt your regularly schedule...
9090,Googles Zeiger a physician never reported pote...,NaN,No emotion toward brand or product,NAN,Googles Zeiger a physician never reported pote...
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product,Apple,Some Verizon iPhone customers complained their...
